# Introduction

In [1]:
import sympy as sy
import numpy as np
A,Av,I,L,J,G,E,x= sy.symbols('A,Av,I,L,J,G,E,x')

Stiffness matrix can be derived by inverting the flexibility matrix and applying a series of transformation. Flexibility coefficients can be determined through principle of virtual works and applying series of unit loads. Refer to Chapter 4 of MASTAN textbook for more info. A shortcut approach was taken since equilibrium matrix can be written for the entire determinate structure
$$
[d]=
\begin{bmatrix}
d_{33} & d_{34}\\
d_{43} & d_{44}\\
\end{bmatrix}
$$

$$
[T]=
\begin{bmatrix}
-1&-L&1&0\\
0&-1&0&1
\end{bmatrix}
$$

$$
[K] = [T]^T [d]^{-1} [T]
$$

# Without shear deformation

In [2]:
d33 = sy.integrate((L-x)**2/E/I,[x,0,L])
d44 = sy.integrate(1/E/I,[x,0,L])
d34 = sy.integrate((L-x)/E/I,[x,0,L])
d43 = d34 #maxwell betti reciprocal theorem

In [3]:
d_noshear = sy.Matrix([
    [d33,d34],
    [d43,d44]
])
d_noshear

Matrix([
[L**3/(3*E*I), L**2/(2*E*I)],
[L**2/(2*E*I),      L/(E*I)]])

In [4]:
T=sy.Matrix([
    [-1,-L,1,0],
    [0,-1,0,1]
])
K_noshear = T.transpose() * d_noshear.inv() * T
K_noshear

Matrix([
[ 12*E*I/L**3,  6*E*I/L**2, -12*E*I/L**3,  6*E*I/L**2],
[  6*E*I/L**2,     4*E*I/L,  -6*E*I/L**2,     2*E*I/L],
[-12*E*I/L**3, -6*E*I/L**2,  12*E*I/L**3, -6*E*I/L**2],
[  6*E*I/L**2,     2*E*I/L,  -6*E*I/L**2,     4*E*I/L]])

# With shear deformation
To include shear deformation, integrate shear strain as well. However, shear deformation only exists on the d33 term. The integral int(vV/GA) is zero for the other terms.

In [5]:
d33 = sy.integrate((L-x)**2/E/I,[x,0,L]) + sy.integrate(1/G/Av,[x,0,L]) 
d44 = sy.integrate(1/E/I,[x,0,L])
d34 = sy.integrate((L-x)/E/I,[x,0,L])
d43 = d34

In [6]:
d_shear = sy.Matrix([
    [d33,d34],
    [d43,d44]
])
d_shear

Matrix([
[L**3/(3*E*I) + L/(Av*G), L**2/(2*E*I)],
[           L**2/(2*E*I),      L/(E*I)]])

In [7]:
T=sy.Matrix([
    [-1,-L,1,0],
    [0,-1,0,1]
])
K_shear = T.transpose() * d_shear.inv() * T
sy.simplify(K_shear)

Matrix([
[ 12*Av*E*G*I/(L*(Av*G*L**2 + 12*E*I)),                    6*Av*E*G*I/(Av*G*L**2 + 12*E*I), -12*Av*E*G*I/(L*(Av*G*L**2 + 12*E*I)),                    6*Av*E*G*I/(Av*G*L**2 + 12*E*I)],
[      6*Av*E*G*I/(Av*G*L**2 + 12*E*I), 4*E*I*(Av*G*L**2 + 3*E*I)/(L*(Av*G*L**2 + 12*E*I)),      -6*Av*E*G*I/(Av*G*L**2 + 12*E*I), 2*E*I*(Av*G*L**2 - 6*E*I)/(L*(Av*G*L**2 + 12*E*I))],
[-12*Av*E*G*I/(L*(Av*G*L**2 + 12*E*I)),                   -6*Av*E*G*I/(Av*G*L**2 + 12*E*I),  12*Av*E*G*I/(L*(Av*G*L**2 + 12*E*I)),                   -6*Av*E*G*I/(Av*G*L**2 + 12*E*I)],
[      6*Av*E*G*I/(Av*G*L**2 + 12*E*I), 2*E*I*(Av*G*L**2 - 6*E*I)/(L*(Av*G*L**2 + 12*E*I)),      -6*Av*E*G*I/(Av*G*L**2 + 12*E*I), 4*E*I*(Av*G*L**2 + 3*E*I)/(L*(Av*G*L**2 + 12*E*I))]])

Note that even though shear deformation is only application for DOF33, it bled into the other stiffness terms after inversion.

Obviously the above form is not very manageable. So textbooks usually write it in terms of another variables. Here is one in terms of theta. Let's verify that they are indeed the same since everything is a algebraic mess right now.

In [8]:
theta = 12*E*I/G/Av/L/L
k_shear_alternative = sy.Matrix([
    [12*E*I/L**3/(1+theta),6*E*I/L/L/(1+theta),-12*E*I/L**3/(1+theta),6*E*I/L/L/(1+theta)],
    [6*E*I/L/L/(1+theta),(4+theta)*E*I/L/(1+theta),-6*E*I/L/L/(1+theta),(2-theta)*E*I/L/(1+theta)],
    [-12*E*I/L**3/(1+theta),-6*E*I/L/L/(1+theta),12*E*I/L**3/(1+theta),-6*E*I/L/L/(1+theta)],
    [6*E*I/L/L/(1+theta),(2-theta)*E*I/L/(1+theta),-6*E*I/L/L/(1+theta),(4+theta)*E*I/L/(1+theta)]
])
k_shear_alternative

Matrix([
[ 12*E*I/(L**3*(1 + 12*E*I/(Av*G*L**2))),                     6*E*I/(L**2*(1 + 12*E*I/(Av*G*L**2))), -12*E*I/(L**3*(1 + 12*E*I/(Av*G*L**2))),                     6*E*I/(L**2*(1 + 12*E*I/(Av*G*L**2)))],
[  6*E*I/(L**2*(1 + 12*E*I/(Av*G*L**2))), E*I*(4 + 12*E*I/(Av*G*L**2))/(L*(1 + 12*E*I/(Av*G*L**2))),  -6*E*I/(L**2*(1 + 12*E*I/(Av*G*L**2))), E*I*(2 - 12*E*I/(Av*G*L**2))/(L*(1 + 12*E*I/(Av*G*L**2)))],
[-12*E*I/(L**3*(1 + 12*E*I/(Av*G*L**2))),                    -6*E*I/(L**2*(1 + 12*E*I/(Av*G*L**2))),  12*E*I/(L**3*(1 + 12*E*I/(Av*G*L**2))),                    -6*E*I/(L**2*(1 + 12*E*I/(Av*G*L**2)))],
[  6*E*I/(L**2*(1 + 12*E*I/(Av*G*L**2))), E*I*(2 - 12*E*I/(Av*G*L**2))/(L*(1 + 12*E*I/(Av*G*L**2))),  -6*E*I/(L**2*(1 + 12*E*I/(Av*G*L**2))), E*I*(4 + 12*E*I/(Av*G*L**2))/(L*(1 + 12*E*I/(Av*G*L**2)))]])

In [9]:
sy.simplify(K_shear - k_shear_alternative)

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

Therefore they are indeed the same!

# Comparison With a W14x120

Let's see how much of a difference inclusion of shear deformation makes! Try 25ft and 15ft which represent typical beam and column span, respectively

In [10]:
def percentagediff(A,B):
    diff_matrix=np.zeros([4,4])
    for i in range(4):
        for j in range(4):
            tempB=float(B[i,j])
            tempA=float(A[i,j])
            diff_matrix[i,j] = (tempB-tempA)/tempB*100
    print(diff_matrix)

### 25 ft

In [11]:
AA=sy.N((K_shear.subs([
    (A,35.3),(Av,8.55),(I,1380),(J,9.37),(E,29000),(G,11154),(L,300)
])),6)
AA

Matrix([
[ 16.8442,  2526.63, -16.8442,  2526.63],
[ 2526.63, 512394.0, -2526.63, 245594.0],
[-16.8442, -2526.63,  16.8442, -2526.63],
[ 2526.63, 245594.0, -2526.63, 512394.0]])

In [12]:
BB=sy.N((K_noshear.subs([
    (A,35.3),(Av,8.55),(I,1380),(J,9.37),(E,29000),(G,11154),(L,300)
])),6)
BB

Matrix([
[ 17.7867,   2668.0, -17.7867,   2668.0],
[  2668.0, 533600.0,  -2668.0, 266800.0],
[-17.7867,  -2668.0,  17.7867,  -2668.0],
[  2668.0, 266800.0,  -2668.0, 533600.0]])

In [13]:
percentagediff(AA,BB)

[[5.29876886 5.29876663 5.29876886 5.29876663]
 [5.29876663 3.97407937 5.29876663 7.94814702]
 [5.29876886 5.29876663 5.29876886 5.29876663]
 [5.29876663 7.94814702 5.29876663 3.97407937]]


### 15 ft

In [14]:
AA=sy.N((K_shear.subs([
    (A,35.3),(Av,8.55),(I,1380),(J,9.37),(E,29000),(G,11154),(L,180)
])),6)
AA

Matrix([
[ 71.2688,  6414.19, -71.2688,  6414.19],
[ 6414.19, 799611.0, -6414.19, 354944.0],
[-71.2688, -6414.19,  71.2688, -6414.19],
[ 6414.19, 354944.0, -6414.19, 799611.0]])

In [15]:
BB=sy.N((K_noshear.subs([
    (A,35.3),(Av,8.55),(I,1380),(J,9.37),(E,29000),(G,11154),(L,180)
])),6)
BB

Matrix([
[ 82.3457,  7411.11, -82.3457,  7411.11],
[ 7411.11, 889333.0, -7411.11, 444667.0],
[-82.3457, -7411.11,  82.3457, -7411.11],
[ 7411.11, 444667.0, -7411.11, 889333.0]])

In [16]:
percentagediff(AA,BB)

[[13.45163979 13.45165317 13.45163979 13.45165317]
 [13.45165317 10.08873641 13.45165317 20.17747282]
 [13.45163979 13.45165317 13.45163979 13.45165317]
 [13.45165317 20.17747282 13.45165317 10.08873641]]


Seems like the difference can be upwards of 13%! Also notice that the percentage difference is same for the shear DOFs, ii moment DOFs, and ij moment DOFs

Let's try a less stocky member. Maybe a W12x26

In [17]:
# 15 ft
AA=sy.N((K_shear.subs([
    (A,35.3),(Av,2.8),(I,204),(J,0.3),(E,29000),(G,11154),(L,180)
])),6)
BB=sy.N((K_noshear.subs([
    (A,35.3),(Av,2.8),(I,204),(J,0.3),(E,29000),(G,11154),(L,180)
])),6)
percentagediff(AA,BB)

[[6.55583122 6.55583996 6.55583122 6.55583996]
 [6.55583996 4.91687174 6.55583996 9.83374349]
 [6.55583122 6.55583996 6.55583122 6.55583996]
 [6.55583996 9.83374349 6.55583996 4.91687174]]


In [18]:
# 25 ft
AA=sy.N((K_shear.subs([
    (A,35.3),(Av,2.8),(I,204),(J,0.3),(E,29000),(G,11154),(L,300)
])),6)
BB=sy.N((K_noshear.subs([
    (A,35.3),(Av,2.8),(I,204),(J,0.3),(E,29000),(G,11154),(L,300)
])),6)
percentagediff(AA,BB)

[[2.46346103 2.46347408 2.46346103 2.46347408]
 [2.46347408 1.84760475 2.46347408 3.69518969]
 [2.46346103 2.46347408 2.46346103 2.46347408]
 [2.46347408 3.69518969 2.46347408 1.84760475]]


Therefore, it seems like shear deformation is quite important for short stocky elements such as some typical columns used in moment frames

# Summary

The same principle applies to bending about the minor axis. However, there must be a few sign changes to be consistent with right hand rule.

once we derive the bending terms about z, bending terms about y, the axial terms and torsional terms are trivial. Then everything can be assembled into a 3D overall stiffness matrix as shown below. Note how they are all decoupled. Red indicate bending about major axis, blue indicate bending about minor axis, axial and torsion terms are self-evident. 

Note that the stiffness matrix below DOES NOT include shear deformation. See one that does include shear deformation here: http://people.duke.edu/~hpgavin/cee421/frame-finite-def.pdf

<img src="../imgs/12x12 stiffness matrix.PNG" style="width: 600px">

# Moment Releases

As far as I can tell, the above approach does not work for moment-released members. The rotation DOF at the released end is dependent on the other DOFs. In other words, we must know the other DOFs before we can calculate the released DOF.

From most matrix analysis textbooks, the stiffness matrix of a beam element with one of the end pinned is given as the following:

DOF1 = beginning node shear  
DOF2 = beginning node moment  
DOF3 = end node shear  
DOF4 = end node moment  

In [19]:
K_bpin = sy.Matrix([
    [3*E*I/L**3,0,-3*E*I/L**3,3*E*I/L**2],
    [0,0,0,0],
    [-3*E*I/L**3,0,3*E*I/L**2,-3*E*I/L**2],
    [3*E*I/L**2,0,-3*E*I/L**2,3*E*I/L]
])
K_bpin

Matrix([
[ 3*E*I/L**3, 0, -3*E*I/L**3,  3*E*I/L**2],
[          0, 0,           0,           0],
[-3*E*I/L**3, 0,  3*E*I/L**2, -3*E*I/L**2],
[ 3*E*I/L**2, 0, -3*E*I/L**2,     3*E*I/L]])

In [20]:
K_epin = sy.Matrix([
    [3*E*I/L**3,3*E*I/L**2,-3*E*I/L**3,0],
    [3*E*I/L**2,3*E*I/L,-3*E*I/L**2,0],
    [-3*E*I/L**3,-3*E*I/L**2,3*E*I/L**3,0],
    [0,0,0,0]
])
K_epin

Matrix([
[ 3*E*I/L**3,  3*E*I/L**2, -3*E*I/L**3, 0],
[ 3*E*I/L**2,     3*E*I/L, -3*E*I/L**2, 0],
[-3*E*I/L**3, -3*E*I/L**2,  3*E*I/L**3, 0],
[          0,           0,           0, 0]])

If both ends are pinned, the member effectively becomes a truss element. Truss elements cannot carry moment nor shear and the stiffness matrix becomes:

DOF1 = beginning node axial  
DOF2 = end node axial 

In [21]:
K_bepin = sy.Matrix([
    [E*A/L,-E*A/L],
    [-E*A/L,E*A/L]
])
K_bepin

Matrix([
[ A*E/L, -A*E/L],
[-A*E/L,  A*E/L]])

However, these matrices do not include shear deformation

# Deriving elements stiffness with shear deformation and beginning node pinned

Let's write down the entire force-displacement relationship from our stiffness matrix above by writing out the matrix multiplication. See chapter 7 of Kassimali textbook for approach. Beam element DOF 2,3,5,6. Note DOF 1 and 4 are axial and was left out for brevity. Vb,Ve,Mb,Me are member fixed-end forces from uniform load, temperature, lack-of-fit, and support settlement

In [22]:
Kij=sy.simplify(K_shear)
u2,u3,u5,u6,Vb,Mb,Ve,Me= sy.symbols('u2,u3,u5,u6,Vb,Mb,Ve,Me')

Q2 = (Kij[0,0]*u2    +Kij[0,1]*u3    -Kij[0,2]*u5    +Kij[0,3]*u6) + Vb
Q3 = (Kij[1,0]*u2    +Kij[1,1]*u3    -Kij[1,2]*u5    +Kij[1,3]*u6) + Mb
Q5 = (Kij[2,0]*u2    -Kij[2,1]*u3    +Kij[2,2]*u5    -Kij[2,3]*u6) + Ve
Q6 = (Kij[3,0]*u2    +Kij[3,1]*u3    -Kij[3,2]*u5    +Kij[3,3]*u6) + Me

Q3 is 0 if beginning node is released. Set Q3 = 0 and then solve for u3. Based on the Kassimali text:

In [23]:
u3_expr = sy.solve(Q3,u3)[0]
u3_expr

-(2*Av*E*G*I*L**2*u6 + 6*Av*E*G*I*L*u2 + 6*Av*E*G*I*L*u5 + Av*G*L**3*Mb - 12*E**2*I**2*u6 + 12*E*I*L*Mb)/(4*E*I*(Av*G*L**2 + 3*E*I))

 ### Subbing u3 into the equation for Q2, Q5, and Q6 gives us:

In [35]:
sy.simplify(Q2.subs([(u3,u3_expr)]))

(3*Av*E*G*I*L*u6 + 3*Av*E*G*I*u2 + 3*Av*E*G*I*u5 + Av*G*L**3*Vb - 3*Av*G*L**2*Mb/2 + 3*E*I*L*Vb)/(L*(Av*G*L**2 + 3*E*I))

In [38]:
sy.simplify(sy.N(Q5.subs([(u3,u3_expr)])),3)

(0.5*Av*E*G*I*L*u6*(Av*G*L**2 + 3.0*E*I) + 1.0*Av*E*G*I*(-u2 + u5)*(Av*G*L**2 + 3.0*E*I) - 0.125*Av*G*L*(2.0*Av*E*G*I*L**2*u6 + 6.0*Av*E*G*I*L*u2 + 6.0*Av*E*G*I*L*u5 + Av*G*L**3*Mb - 12.0*E**2*I**2*u6 + 12.0*E*I*L*Mb) + 1.0*L*Ve*(0.0833333333333333*Av*G*L**2 + E*I)*(Av*G*L**2 + 3.0*E*I))/(L*(0.0833333333333333*Av*G*L**2 + E*I)*(Av*G*L**2 + 3.0*E*I))

In [36]:
sy.simplify(Q6.subs([(u3,u3_expr)]))

(3*Av*E*G*I*L*u6 + 3*Av*E*G*I*u2 + 3*Av*E*G*I*u5 - Av*G*L**2*Mb/2 + Av*G*L**2*Me + 3*E*I*Mb + 3*E*I*Me)/(Av*G*L**2 + 3*E*I)

Note how the Fixed-end forces also change and becomes a function of stiffness. Let's do some quick comparisons

# Comparison of K
For brevity lets compare a few terms

In [27]:
k11 = 3*E*I/L**3
k11

3*E*I/L**3

If we were to include shear deformation as indicated above:

In [28]:
X = Av*G*L*L+3*E*I
k11_shear = 3*Av*E*G*I/L/X
k11_shear

3*Av*E*G*I/(L*(Av*G*L**2 + 3*E*I))

In [29]:
test1=k11_shear.subs([(Av,8.55),(I,1380),(E,29000),(G,11154),(L,180)])
test1

19.8164350418544

In [30]:
test2=sy.N(k11.subs([(I,1380),(E,29000),(L,180)]),6)
test2

20.5864

In [31]:
(test2 - test1)/test2

0.0374024808162979

Notice how with the same W14x120 member and 15 ft length, the percentage difference is only 3%, compared to 13% of the fixed-fixed elements

# Comparison of FEF

In [40]:
w=sy.symbols('w')
FEF = 3*w*L/8
FEF

3*L*w/8

In [41]:
Vb=w*L/2
Mb=w*L*L/12

FEF_shear= (Av*G*L*L*L*Vb + 3*E*I*L*Vb - 3*Av*G*L*L*Mb/2)/X/L

sy.simplify(FEF_shear)

3*L*w*(Av*G*L**2 + 4*E*I)/(8*(Av*G*L**2 + 3*E*I))

Notice how the second term has a ratio of stiffnesses. Let's see the percentage difference

In [42]:
test3=FEF.subs([(w,0.5),(L,180)])
test3

33.7500000000000

In [43]:
test4=FEF_shear.subs([(w,0.5),(L,180),(Av,8.55),(I,1380),(E,29000),(G,11154)])
test4

34.1707787514904

In [44]:
(test3 - test4)/test3*100

-1.24675185626773

The difference is about 1.2% which is very small!